In [1]:
import pandas as pd
import folium
from collections import Counter
from pathlib import Path

In [2]:
gtfs_path = Path("..") / "data" / "raw" / "budapest_gtfs"

In [3]:
# Load GTFS core files
stops = pd.read_csv(gtfs_path / "stops.txt")
routes = pd.read_csv(gtfs_path / "routes.txt")
trips = pd.read_csv(gtfs_path / "trips.txt")
stop_times = pd.read_csv(gtfs_path / "stop_times.txt")

In [4]:
stops.head()

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding
0,002133,"Örs vezér tere M+H, déli tárolótér",47.500366,19.135700,002133,NaN,NaN,NaN,NaN
1,002138,Kőbánya alsó vasútállomás,47.483139,19.127891,002138,NaN,NaN,NaN,2.0
2,003002,Puskás Ferenc Stadion M,47.500368,19.103406,003002,NaN,NaN,NaN,NaN
3,004716,"ÉD metró járműtelep,porta",47.469651,19.129090,004716,NaN,NaN,NaN,2.0
4,004948,Metró ÉD járműtelep (kapu),47.465239,19.142612,004948,NaN,NaN,NaN,NaN


In [5]:
routes.head()

,agency_id,route_id,route_short_name,route_long_name,route_type,route_desc,route_color,route_text_color,route_sort_order
0,BKK,0050,5,NaN,3,"Pasaréti tér / Rákospalota, Kossuth utca",009EE3,FFFFFF,20
1,BKK,0070,7,NaN,3,"Albertfalva vasútállomás / Újpalota, Nyírpalot...",009EE3,FFFFFF,24
2,BKK,0075,7E,NaN,3,"Blaha Lujza tér M / Újpalota, Nyírpalota út",009EE3,FFFFFF,25
3,BKK,0078,7G,NaN,3,"Cinkotai autóbuszgarázs / Újpalota, Nyírpalota út",009EE3,FFFFFF,26
4,BKK,0085,8E,NaN,3,"Kelenföld vasútállomás M / Újpalota, Nyírpalot...",009EE3,FFFFFF,27


In [6]:
trips.head()

,route_id,trip_id,service_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,8140,C0418310,C04183AHPMAA-011,"Soroksár, Molnár-sziget",0,C04183_8140_1_5,Y701,1.0,1.0
1,8140,C0418311,C04183AHPMAA-011,Csepel-Királyerdő,1,C04183_8140_1_6,Y702,1.0,1.0
2,8140,C0418312,C04183AHPMAA-011,"Soroksár, Molnár-sziget",0,C04183_8140_1_7,Y701,1.0,1.0
3,8140,C0418313,C04183AHPMAA-011,Csepel-Királyerdő,1,C04183_8140_1_8,Y702,1.0,1.0
4,8140,C0418314,C04183AHPMAA-011,"Soroksár, Molnár-sziget",0,C04183_8140_1_9,Y701,1.0,1.0


In [7]:
stop_times.head()

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,C0418310,F04181,08:03:00,08:03:00,0,NaN,NaN,NaN,0.0
1,C0418310,F04526,08:06:00,08:06:00,1,NaN,NaN,NaN,160.0
2,C0418311,F04526,08:30:00,08:30:00,0,NaN,NaN,NaN,0.0
3,C0418311,F04181,08:33:00,08:33:00,1,NaN,NaN,NaN,160.0
4,C0418312,F04181,08:33:00,08:33:00,0,NaN,NaN,NaN,0.0


In [9]:
# Clean data
stops = stops.dropna(subset=["stop_lat", "stop_lon"])
stops["stop_name"] = stops["stop_name"].str.strip()

In [10]:
# Map all tram/bus stops in Budapest
budapest_map = folium.Map(location=[47.4979, 19.0402], zoom_start=12)
for _, row in stops.iterrows():
    folium.CircleMarker(
        location=[row["stop_lat"], row["stop_lon"]],
        radius=2,
        popup=row["stop_name"],
        color="blue",
        fill=True,
        fill_opacity=0.5
    ).add_to(budapest_map)

In [ ]:
# Find a specific route, e.g., line 4-6 tram
route_ids = routes[routes["route_short_name"].isin(["4", "6"])]["route_id"].tolist()
trips_4_6 = trips[trips["route_id"].isin(route_ids)]
stop_ids_4_6 = stop_times[stop_times["trip_id"].isin(trips_4_6["trip_id"])]["stop_id"].unique()
stops_4_6 = stops[stops["stop_id"].isin(stop_ids_4_6)]

In [12]:
# Add 4-6 tram stops to map in red
for _, row in stops_4_6.iterrows():
    folium.CircleMarker(
        location=[row["stop_lat"], row["stop_lon"]],
        radius=3,
        popup=row["stop_name"],
        color="red",
        fill=True,
        fill_opacity=0.8
    ).add_to(budapest_map)

In [13]:
# Identify most connected stops (hubs) by counting stop_id occurrences in stop_times
stop_counts = Counter(stop_times["stop_id"])
hub_data = pd.DataFrame(stop_counts.items(), columns=["stop_id", "count"]).sort_values(by="count", ascending=False)
hubs = stops[stops["stop_id"].isin(hub_data.head(10)["stop_id"])]

In [14]:
# Add hubs to map in green
for _, row in hubs.iterrows():
    folium.CircleMarker(
        location=[row["stop_lat"], row["stop_lon"]],
        radius=5,
        popup=f"{row['stop_name']} (hub)",
        color="green",
        fill=True,
        fill_opacity=1.0
    ).add_to(budapest_map)

In [23]:
budapest_map.save("../data/budapest_transport_map.html")